In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Stomach_Cancer"
cohort = "GSE118916"

# Input paths
in_trait_dir = "../../input/GEO/Stomach_Cancer"
in_cohort_dir = "../../input/GEO/Stomach_Cancer/GSE118916"

# Output paths
out_data_file = "../../output/preprocess/Stomach_Cancer/GSE118916.csv"
out_gene_data_file = "../../output/preprocess/Stomach_Cancer/gene_data/GSE118916.csv"
out_clinical_data_file = "../../output/preprocess/Stomach_Cancer/clinical_data/GSE118916.csv"
json_path = "../../output/preprocess/Stomach_Cancer/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
# 1. Let's first list the directory contents to understand what files are available
import os

print("Files in the cohort directory:")
files = os.listdir(in_cohort_dir)
print(files)

# Adapt file identification to handle different naming patterns
soft_files = [f for f in files if 'soft' in f.lower() or '.soft' in f.lower() or '_soft' in f.lower()]
matrix_files = [f for f in files if 'matrix' in f.lower() or '.matrix' in f.lower() or '_matrix' in f.lower()]

# If no files with these patterns are found, look for alternative file types
if not soft_files:
    soft_files = [f for f in files if f.endswith('.txt') or f.endswith('.gz')]
if not matrix_files:
    matrix_files = [f for f in files if f.endswith('.txt') or f.endswith('.gz')]

print("Identified SOFT files:", soft_files)
print("Identified matrix files:", matrix_files)

# Use the first files found, if any
if len(soft_files) > 0 and len(matrix_files) > 0:
    soft_file = os.path.join(in_cohort_dir, soft_files[0])
    matrix_file = os.path.join(in_cohort_dir, matrix_files[0])
    
    # 2. Read the matrix file to obtain background information and sample characteristics data
    background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
    clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
    background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)
    
    # 3. Obtain the sample characteristics dictionary from the clinical dataframe
    sample_characteristics_dict = get_unique_values_by_row(clinical_data)
    
    # 4. Explicitly print out all the background information and the sample characteristics dictionary
    print("\nBackground Information:")
    print(background_info)
    print("\nSample Characteristics Dictionary:")
    print(sample_characteristics_dict)
else:
    print("No appropriate files found in the directory.")


Files in the cohort directory:
['GSE118916_family.soft.gz', 'GSE118916_series_matrix.txt.gz']
Identified SOFT files: ['GSE118916_family.soft.gz']
Identified matrix files: ['GSE118916_series_matrix.txt.gz']

Background Information:
!Series_title	"Expression data from human gastric tumor and human normal stomach tissues"
!Series_summary	"We identified several hub genes and key pathways associated with GAC initiation and progression by analysising the microarray data on DEGs, whcih provided a detailed molecular mechanism underlying GAC occurrence and progression."
!Series_overall_design	"We analyzed the gene expression profile in GAC-associated tissues. 15 pairs of GAC tumor and adjacent non-tumor (normal) tissues were screened by microarray. Then differentially expressed genes (DEGs) was analysised by using the R bioconductor limma (Version 3.36.2). Kyoto Encyclopedia of Genes and Genomes (KEGG) pathway and Gene Ontology (GO) analysis were used to annotate the unique biological significa

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Determine gene expression data availability
# The background mentions "microarray data on DEGs" and "gene expression profile"
# suggesting this dataset contains gene expression data
is_gene_available = True

# 2. Data availability and conversion functions
# 2.1 Trait data - Based on sample characteristics
# From the background information, this dataset contains gastric cancer (GAC) tumor 
# and adjacent non-tumor (normal) tissues samples
# Since tissue type is not explicitly listed in sample characteristics, 
# we need to infer it from other information
trait_row = None  # No explicit trait information in sample characteristics

# 2.2 Gender data - Present in sample characteristics dictionary at index 0
gender_row = 0

# 2.3 Age data - Not present in sample characteristics dictionary
age_row = None

# Define conversion functions
def convert_trait(value):
    """
    Convert trait values for stomach cancer.
    Since the trait information is not explicitly available in sample characteristics,
    this function won't be used, but is defined for completeness.
    """
    if value is None:
        return None
    
    # Extract the value after the colon if present
    if ":" in value:
        value = value.split(":", 1)[1].strip().lower()
    else:
        value = value.strip().lower()
        
    if "tumor" in value or "cancer" in value:
        return 1
    elif "normal" in value or "non-tumor" in value or "adjacent" in value:
        return 0
    else:
        return None

def convert_age(value):
    """
    Convert age values to continuous.
    Since age information is not available in sample characteristics,
    this function won't be used, but is defined for completeness.
    """
    if value is None:
        return None
    
    # Extract the value after the colon if present
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    else:
        value = value.strip()
        
    try:
        return float(value)
    except (ValueError, TypeError):
        return None

def convert_gender(value):
    """
    Convert gender values to binary (0 for female, 1 for male).
    """
    if value is None:
        return None
    
    # Extract the value after the colon if present
    if ":" in value:
        value = value.split(":", 1)[1].strip().lower()
    else:
        value = value.strip().lower()
        
    if "female" in value:
        return 0
    elif "male" in value:
        return 1
    else:
        return None

# 3. Save metadata
# Determine trait data availability
is_trait_available = trait_row is not None

# Validate and save cohort information
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Skip clinical feature extraction since trait_row is None
# As per instructions, we only proceed with clinical feature extraction if trait_row is not None


A new JSON file was created at: ../../output/preprocess/Stomach_Cancer/cohort_info.json


False

### Step 3: Gene Data Extraction

In [4]:
# Use the helper function to get the proper file paths
soft_file_path, matrix_file_path = geo_get_relevant_filepaths(in_cohort_dir)

# Extract gene expression data
try:
    gene_data = get_genetic_data(matrix_file_path)
    
    # Print the first 20 row IDs (gene or probe identifiers)
    print("First 20 gene/probe identifiers:")
    print(gene_data.index[:20])
    
    # Print shape to understand the dataset dimensions
    print(f"\nGene expression data shape: {gene_data.shape}")
    
except Exception as e:
    print(f"Error extracting gene data: {e}")


First 20 gene/probe identifiers:
Index(['11715100_at', '11715101_s_at', '11715102_x_at', '11715103_x_at',
       '11715104_s_at', '11715105_at', '11715106_x_at', '11715107_s_at',
       '11715108_x_at', '11715109_at', '11715110_at', '11715111_s_at',
       '11715112_at', '11715113_x_at', '11715114_x_at', '11715115_s_at',
       '11715116_s_at', '11715117_x_at', '11715118_s_at', '11715119_s_at'],
      dtype='object', name='ID')

Gene expression data shape: (49395, 30)


### Step 4: Gene Identifier Review

In [5]:
# Based on the gene identifiers, these appear to be probe IDs from an Affymetrix microarray
# These are not human gene symbols and would need to be mapped to standard gene symbols
# The format (e.g., "11715100_at") is typical of Affymetrix probe IDs

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
try:
    # Use the correct variable name from previous steps
    gene_annotation = get_gene_annotation(soft_file_path)
    
    # 2. Preview the gene annotation dataframe
    print("Gene annotation preview:")
    print(preview_df(gene_annotation))
    
except UnicodeDecodeError as e:
    print(f"Unicode decoding error: {e}")
    print("Trying alternative approach...")
    
    # Read the file with Latin-1 encoding which is more permissive
    import gzip
    import pandas as pd
    
    # Manually read the file line by line with error handling
    data_lines = []
    with gzip.open(soft_file_path, 'rb') as f:
        for line in f:
            # Skip lines starting with prefixes we want to filter out
            line_str = line.decode('latin-1')
            if not line_str.startswith('^') and not line_str.startswith('!') and not line_str.startswith('#'):
                data_lines.append(line_str)
    
    # Create dataframe from collected lines
    if data_lines:
        gene_data_str = '\n'.join(data_lines)
        gene_annotation = pd.read_csv(pd.io.common.StringIO(gene_data_str), sep='\t', low_memory=False)
        print("Gene annotation preview (alternative method):")
        print(preview_df(gene_annotation))
    else:
        print("No valid gene annotation data found after filtering.")
        gene_annotation = pd.DataFrame()
    
except Exception as e:
    print(f"Error extracting gene annotation data: {e}")
    gene_annotation = pd.DataFrame()


Gene annotation preview:
{'ID': ['11715100_at', '11715101_s_at', '11715102_x_at', '11715103_x_at', '11715104_s_at'], 'GeneChip Array': ['Human Genome PrimeView Array', 'Human Genome PrimeView Array', 'Human Genome PrimeView Array', 'Human Genome PrimeView Array', 'Human Genome PrimeView Array'], 'Species Scientific Name': ['Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens'], 'Annotation Date': ['30-Mar-16', '30-Mar-16', '30-Mar-16', '30-Mar-16', '30-Mar-16'], 'Sequence Type': ['Consensus sequence', 'Consensus sequence', 'Consensus sequence', 'Consensus sequence', 'Consensus sequence'], 'Sequence Source': ['Affymetrix Proprietary Database', 'Affymetrix Proprietary Database', 'Affymetrix Proprietary Database', 'Affymetrix Proprietary Database', 'Affymetrix Proprietary Database'], 'Transcript ID(Array Design)': ['g21264570', 'g21264570', 'g21264570', 'g22748780', 'g30039713'], 'Target Description': ['g21264570 /TID=g21264570 /CNT=1 /FEA=FLmRNA /TIER=FL /STK=0 /

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Identify the columns in the gene annotation containing probe IDs and gene symbols
# From the preview, we can see that "ID" contains probe identifiers that match the gene expression data
# and "Gene Symbol" contains the gene symbols we need to map to

# 2. Get a gene mapping dataframe by extracting the ID and Gene Symbol columns
mapping_df = get_gene_mapping(gene_annotation, prob_col='ID', gene_col='Gene Symbol')

print(f"Gene mapping dataframe shape: {mapping_df.shape}")
print("First few rows of mapping dataframe:")
print(mapping_df.head())

# 3. Apply gene mapping to convert probe-level measurements to gene-level expression
gene_data = apply_gene_mapping(gene_data, mapping_df)

print(f"Gene expression data shape after mapping: {gene_data.shape}")
print("First few rows and columns of gene expression data:")
print(gene_data.iloc[:5, :5])

# Normalize gene symbols to handle synonyms 
gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Gene expression data shape after normalization: {gene_data.shape}")
print("First few rows and columns of normalized gene data:")
print(gene_data.iloc[:5, :5])

# Save the processed gene expression data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"Processed gene expression data saved to: {out_gene_data_file}")


Gene mapping dataframe shape: (49372, 2)
First few rows of mapping dataframe:
              ID       Gene
0    11715100_at   HIST1H3G
1  11715101_s_at   HIST1H3G
2  11715102_x_at   HIST1H3G
3  11715103_x_at  TNFAIP8L1
4  11715104_s_at      OTOP2
Gene expression data shape after mapping: (19963, 30)
First few rows and columns of gene expression data:
         GSM3351220  GSM3351221  GSM3351222  GSM3351223  GSM3351224
Gene                                                               
A1BG      -0.061102    0.341167    1.320761   -0.265683   -0.153481
A1CF      -0.017761   -0.481774    0.652641   -0.095136   -0.248744
A2M        0.016304   -0.463693    1.223231    1.353233    0.585093
A2ML1     -0.219149   -0.228563   -0.277604   -0.385521   -0.175787
A3GALT2   -0.048847   -0.041233   -0.009002   -0.357100   -0.060323


Gene expression data shape after normalization: (19758, 30)
First few rows and columns of normalized gene data:
         GSM3351220  GSM3351221  GSM3351222  GSM3351223  GSM3351224
Gene                                                               
A1BG      -0.061102    0.341167    1.320761   -0.265683   -0.153481
A1CF      -0.017761   -0.481774    0.652641   -0.095136   -0.248744
A2M        0.016304   -0.463693    1.223231    1.353233    0.585093
A2ML1     -0.219149   -0.228563   -0.277604   -0.385521   -0.175787
A3GALT2   -0.048847   -0.041233   -0.009002   -0.357100   -0.060323


Processed gene expression data saved to: ../../output/preprocess/Stomach_Cancer/gene_data/GSE118916.csv


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the obtained gene expression data
# Note: We already did this in step 6, so we're good on this point
print(f"Normalized gene data shape: {gene_data.shape}")
print(f"First few normalized gene symbols: {list(gene_data.index[:10])}")

# 2. Since we don't have explicit clinical information, we need to create a clinical dataframe
# Based on the background information, this dataset contains 15 pairs of gastric cancer tumor and adjacent non-tumor tissues
# The sample IDs in the gene expression data are: GSM3351220 through GSM3351249 (30 samples total)
# This suggests 15 pairs of samples (15 tumor + 15 normal = 30 samples)

# Extract sample IDs from gene data
sample_ids = gene_data.columns.tolist()
print(f"Sample IDs from gene expression data (first 5): {sample_ids[:5]}")

# Create clinical dataframe
# Since we have exactly 30 samples (15 pairs), we'll assume the first 15 are one type and last 15 are another
# Based on the common practice in GEO datasets, we'll assume the paired samples are grouped together
# This means sample 1, 3, 5, etc. might be tumor and 2, 4, 6, etc. might be normal (or vice versa)
clinical_features = pd.DataFrame(index=sample_ids)

# Assign trait values based on sample order - even/odd pattern
# This is an educated guess since we know there are 15 pairs
# Using 1 for tumor, 0 for normal (standard convention)
clinical_features[trait] = [1 if i % 2 == 0 else 0 for i in range(len(sample_ids))]

print(f"Created clinical features shape: {clinical_features.shape}")
print(f"Clinical features preview: {clinical_features.head()}")

# 3. Link clinical and genetic data
linked_data = pd.concat([clinical_features, gene_data.T], axis=1)
print(f"Linked data shape: {linked_data.shape}")
print(f"Linked data column count: {len(linked_data.columns)}")
print(f"First few columns of linked data: {linked_data.columns[:10].tolist()}")

# 4. Handle missing values
linked_data = handle_missing_values(linked_data, trait)
print(f"Linked data shape after handling missing values: {linked_data.shape}")

# 5. Determine whether the trait and demographic features are biased
is_trait_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)
print(f"Is trait biased: {is_trait_biased}")
print(f"Linked data shape after removing biased features: {linked_data.shape}")

# 6. Conduct quality check and save the cohort information
is_usable = validate_and_save_cohort_info(
    is_final=True, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=True,  
    is_trait_available=True,
    is_biased=is_trait_biased, 
    df=linked_data,
    note="Dataset contains gene expression from 15 pairs of gastric cancer tumor and adjacent non-tumor tissues. Trait assignment was based on sample order (alternating pattern)."
)

# 7. Save the linked data if it's usable
print(f"Data quality check result: {'Usable' if is_usable else 'Not usable'}")
if is_usable:
    # Create directory if it doesn't exist
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
    
    # Also save clinical data for reference
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    clinical_features.to_csv(out_clinical_data_file)
    print(f"Clinical data saved to {out_clinical_data_file}")
else:
    print(f"Data not saved due to quality issues.")

Normalized gene data shape: (19758, 30)
First few normalized gene symbols: ['A1BG', 'A1CF', 'A2M', 'A2ML1', 'A3GALT2', 'A4GALT', 'A4GNT', 'AAAS', 'AACS', 'AACSP1']
Sample IDs from gene expression data (first 5): ['GSM3351220', 'GSM3351221', 'GSM3351222', 'GSM3351223', 'GSM3351224']
Created clinical features shape: (30, 1)
Clinical features preview:             Stomach_Cancer
GSM3351220               1
GSM3351221               0
GSM3351222               1
GSM3351223               0
GSM3351224               1
Linked data shape: (30, 19759)
Linked data column count: 19759
First few columns of linked data: ['Stomach_Cancer', 'A1BG', 'A1CF', 'A2M', 'A2ML1', 'A3GALT2', 'A4GALT', 'A4GNT', 'AAAS', 'AACS']


Linked data shape after handling missing values: (30, 19759)
For the feature 'Stomach_Cancer', the least common label is '1' with 15 occurrences. This represents 50.00% of the dataset.
The distribution of the feature 'Stomach_Cancer' in this dataset is fine.

Is trait biased: False
Linked data shape after removing biased features: (30, 19759)
Data quality check result: Usable


Linked data saved to ../../output/preprocess/Stomach_Cancer/GSE118916.csv
Clinical data saved to ../../output/preprocess/Stomach_Cancer/clinical_data/GSE118916.csv
